In [ ]:
from google.colab import drive
drive.mount('/content/drive2', force_remount=True)
driveHome = "/content/drive2/MyDrive/MasterThesis"

Mounted at /content/drive2


In [ ]:
!pip install imgaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def flushDrive():
    drive.flush_and_unmount()
    drive.mount('/content/drive2', force_remount=True)

In [ ]:
import imgaug.augmenters as iaa

import os
from os.path import join, basename, exists
from pathlib import Path
import cv2
import shutil
import time
import re
from collections import defaultdict
import glob
import numpy as np
import random
from imgaug import imgaug
from datetime import datetime

In [ ]:
localData = "/home/local/"
UcfPath = os.path.join(driveHome, "UCF-SmartSplit")
UcfAugmentExtract = os.path.join(UcfPath, "augmentation-prep")
UcfAugmented = os.path.join(UcfPath, "augmented")

In [ ]:
quantities = [250, 500, 750, 1000, 1500, 2000, 3000]
percentages = [0.1, 0.333, 0.666, 0.9]

In [ ]:
def augment(source, targetDir, operation, quantity):
    if exists(targetDir):
        return

    seq = None
    if operation == "crop":
        seq = iaa.Sequential([
            iaa.Fliplr(0.5),
            iaa.Crop(percent=(0.05, 0.2)),
        ])
    elif operation == "brightness":
        seq = iaa.Sequential([
            iaa.Fliplr(0.5),
            iaa.Add((-40, 40))
        ])
    elif operation == "gaussian-blur":
        seq = iaa.Sequential([
            iaa.Fliplr(0.5),
            iaa.GaussianBlur(sigma=(1, 2.5))
        ])
    elif operation == "greyscale":
        seq = iaa.Sequential([
            iaa.Fliplr(0.5),
            iaa.Grayscale(alpha=(0.5, 1.0))
        ])
    elif operation == "gaussian-noise":
        seq = iaa.Sequential([
            iaa.Fliplr(0.5),
            iaa.AdditiveGaussianNoise(scale=(1, 0.1*255))
        ])
    elif operation == "gaussian-mix":
        seq = iaa.Sequential([
            iaa.Fliplr(0.5),
            iaa.AdditiveGaussianNoise(scale=(1, 0.1*255)),
            iaa.GaussianBlur(sigma=(1, 2.5))
        ])
    elif operation == "greyscale-brightness":
        seq = iaa.Sequential([
            iaa.Fliplr(0.5),
            iaa.Grayscale(alpha=(0.5, 1.0)),
            iaa.Add((-40, 40))
        ])
    elif operation == "mix-net":
        seq = iaa.Sequential([
            iaa.Fliplr(0.5),
            iaa.Affine(
                scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                rotate=(-12, 12),
                shear=(-12, 12),
                mode="edge"
            )
        ])
    elif operation == "mix-weak":
        seq = iaa.Sequential([
            iaa.Fliplr(0.5),
            iaa.Crop(percent=(0.05, 0.2)),
            iaa.GaussianBlur(sigma=(0.5, 2.5))
        ])
    else:
        print("Wrong operation name")

    imageList = glob.glob(join(source, "*.jpg"))
    batch_size = min(quantity, len(imageList))
    exportedImages = 0

    while(exportedImages < quantity):
        imagesToAugment = np.array(
            random.sample( 
                [cv2.cvtColor(cv2.imread(im), cv2.COLOR_BGR2RGB) for im in imageList], 
                batch_size),
            dtype=np.uint8)

        augmentedImages = seq(images = imagesToAugment)

        Path(targetDir).mkdir(parents=True, exist_ok=True)
        for img in augmentedImages:
            filename = join(targetDir, f"augmented_{exportedImages:04d}.jpg")
            if not cv2.imwrite(filename, cv2.cvtColor(img, cv2.COLOR_BGR2RGB)):
                raise Exception("Could not write image")
            exportedImages += 1
            if exportedImages >= quantity:
                break
        
    flushDrive()
    time = datetime.now().strftime("%H:%M:%S")
    print(f"Done: {time}")

In [ ]:
operations = ["gaussian-mix", "greyscale-brightness", "mix-net", "crop", "gaussian-blur", "gaussian-noise", "brightness", "greyscale", "mix-weak"]

In [ ]:
for perc in percentages:
    percentageDir = join(UcfAugmentExtract, str(perc))
    for operation in operations:
        print(f"\n---{operation}---")
        for quantity in quantities:
            quantityDir = join(percentageDir, str(quantity))
            for actionName in os.listdir(quantityDir):
                    source = join(quantityDir, actionName)
                    print(f"\t{perc}---{operation}---{int(quantity)}---{actionName}")
                    target = join(UcfAugmented, perc, operation, quantity, actionName)
                    augment(source, target, operation, int(quantity) - len(os.listdir(source)))

In [ ]:
flushDrive()